# TP4

## Dataset

### Dataset study and objectives

### Choosing the variables

### Dataset cleaning

### Further processing

### Distribution of data in the dataset

## Federated Learning Model

### Model chosen for our task